In [ ]:
# !pip install requests
# !pip install beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup, Tag
import pandas as pd
import re

In [ ]:
!pip install gspread pandas openpyxl

In [ ]:
from google.colab import auth
from google.auth import default
import gspread
import pandas as pd

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
# Create a new spreadsheet (optional, if you don't have one already)
# spreadsheet = gc.create('Product Data from Colab')
# print(f"Spreadsheet created: {spreadsheet.url}")

def update_sheet(df, sheet_name="Sheet1"):
  # Or open an existing spreadsheet by title
  spreadsheet = gc.open('AutoProductList') # actual sheet title

  # Select a worksheet by index (0 for the first sheet) or by name
  # worksheet = spreadsheet.sheet1 # Or spreadsheet.worksheet('Sheet1')
  worksheet = spreadsheet.worksheet(sheet_name)

  # Add headers if they are not already present in the sheet
  worksheet.update([df.columns.values.tolist()] + df.values.tolist())

  print("Data successfully uploaded to Google Sheets!")

In [ ]:
ssiurl='https://ssisports.net/Categories/Index'
url='https://ssisports.net/Categories/List/Binoculars-and-Monoculars'

In [ ]:
# chattanoogashooting
url = "https://chattanoogashooting.com/catalog/search/category_optics"


Status Code: 200


1

In [ ]:
url = "https://www.rothco.com/collection/womens-t-shirts-and-tank-tops-top-6?ss%5Bss_category_hierarchy%5D=Clothing"
content = fetch_url(url)

Status Code: 200


In [ ]:
soup = BeautifulSoup(content, 'html.parser')

In [ ]:
soup.find(class_='product-card')

In [ ]:
api_url = "https://cdn.builder.io/api/v3/content/color-camo-swatches?apiKey=44203814f365498eb9faf08435201e21&limit=8&noTraverse=true&includeRefs=true&omit=meta.componentsUsed&query.data.name.%24in=%5B%22Black%22%2C%22Midnight+Black+Camo%22%2C%22Terracotta%22%2C%22Tiger+Stripe+Camo%22%2C%22Vintage+Teal%22%2C%22Pink+Camo%22%2C%22Woodland+Camo%22%2C%22Ultra+Violet+Camo%22%5D&query.startDate.%24or=%5B%7B%22%24exists%22%3Afalse%7D%2C%7B%22%24lte%22%3A1751048817253%7D%5D&query.endDate.%24or=%5B%7B%22%24exists%22%3Afalse%7D%2C%7B%22%24gte%22%3A1751048817253%7D%5D"

response = requests.get(api_url)

In [ ]:
response

<Response [200]>

In [ ]:
response.json()

{'results': [{'query': [],
   'folders': [],
   'createdDate': 1727893014720,
   'id': 'a936ce8ceb574b459eb9da9110fcb0b6',
   'name': 'Terracotta Swatch',
   'modelId': 'beb91943e94b4340bee4428eeeca4c6e',
   'meta': {'kind': 'data', 'lastPreviewUrl': ''},
   'variations': {},
   'lastUpdated': 1733520520556,
   'testRatio': 1,
   'createdBy': 'n240ODMoFmY7zZ9wwVXvJtM7fi92',
   'lastUpdatedBy': 'n240ODMoFmY7zZ9wwVXvJtM7fi92',
   'data': {'name': 'Terracotta',
    'color-family': [{'name': 'RED'}],
    'swatch': 'https://cdn.builder.io/api/v1/image/assets%2F44203814f365498eb9faf08435201e21%2F321bd2d52a114858b4ab0bd614edb161',
    'colorFamily': [{'name': 'Red'},
     {'name': 'Terracotta'},
     {'name': 'Pink'}]},
   'published': 'published',
   'firstPublished': 1735672235422,
   'rev': 'xvjxu4nv7z8'},
  {'query': [],
   'folders': [],
   'createdDate': 1727893014356,
   'id': '439cd4f8cd3541bdb85e808b90d88004',
   'name': 'Vintage Teal Swatch',
   'modelId': 'beb91943e94b4340bee4428ee

In [ ]:

import json

products_list = response.json().get("results", [])

# You can now work with the products_list
print(f"Found {len(products_list)} products.")
print("First product:", products_list[0])

Found 8 products.
First product: {'query': [], 'folders': [], 'createdDate': 1727893014720, 'id': 'a936ce8ceb574b459eb9da9110fcb0b6', 'name': 'Terracotta Swatch', 'modelId': 'beb91943e94b4340bee4428eeeca4c6e', 'meta': {'kind': 'data', 'lastPreviewUrl': ''}, 'variations': {}, 'lastUpdated': 1733520520556, 'testRatio': 1, 'createdBy': 'n240ODMoFmY7zZ9wwVXvJtM7fi92', 'lastUpdatedBy': 'n240ODMoFmY7zZ9wwVXvJtM7fi92', 'data': {'name': 'Terracotta', 'color-family': [{'name': 'RED'}], 'swatch': 'https://cdn.builder.io/api/v1/image/assets%2F44203814f365498eb9faf08435201e21%2F321bd2d52a114858b4ab0bd614edb161', 'colorFamily': [{'name': 'Red'}, {'name': 'Terracotta'}, {'name': 'Pink'}]}, 'published': 'published', 'firstPublished': 1735672235422, 'rev': 'xvjxu4nv7z8'}


In [ ]:
# Convert the list of dictionaries to a pandas DataFrame
df_builder = pd.DataFrame(products_list)

# Select the desired columns
selected_columns = ['id', 'name', 'data', 'createdDate', 'lastUpdated', 'published']
df_builder_filtered = df_builder[selected_columns].copy()

# Normalize the 'data' column to extract desired fields
df_builder_filtered[['data_name', 'data_swatch']] = df_builder_filtered['data'].apply(lambda x: pd.Series({
    'data_name': x.get('name', None),
    'data_swatch': x.get('swatch', None)
}))

# Drop the original 'data' column if you don't need it anymore
df_builder_filtered.drop('data', axis=1, inplace=True)

# Convert timestamps to datetime objects
df_builder_filtered['createdDate'] = pd.to_datetime(df_builder_filtered['createdDate'], unit='ms')
df_builder_filtered['lastUpdated'] = pd.to_datetime(df_builder_filtered['lastUpdated'], unit='ms')

In [ ]:
# typecase timestamps into str

df_builder_filtered['createdDate'] = df_builder_filtered['createdDate'].astype(str)
df_builder_filtered['lastUpdated'] = df_builder_filtered['lastUpdated'].astype(str)

In [ ]:
# types

df_builder_filtered.dtypes

,0
id,object
name,object
createdDate,object
lastUpdated,object
published,object
data_name,object
data_swatch,object


In [ ]:
df_builder_filtered.head()

,id,name,createdDate,lastUpdated,published,data_name,data_swatch
0,a936ce8ceb574b459eb9da9110fcb0b6,Terracotta Swatch,2024-10-02 18:16:54.720,2024-12-06 21:28:40.556,published,Terracotta,https://cdn.builder.io/api/v1/image/assets%2F4...
1,439cd4f8cd3541bdb85e808b90d88004,Vintage Teal Swatch,2024-10-02 18:16:54.356,2024-12-13 16:42:56.349,published,Vintage Teal,https://cdn.builder.io/api/v1/image/assets%2F4...
2,2570a088991e4f4d93b69f2f87549b3b,Ultra Violet Camo Swatch,2024-10-02 18:16:31.039,2024-12-13 16:42:04.622,published,Ultra Violet Camo,https://cdn.builder.io/api/v1/image/assets%2F4...
3,b777894b0e7c4daba3abf440f2d0d37b,Tiger Stripe Camo Swatch,2024-10-02 18:16:29.185,NaT,published,Tiger Stripe Camo,https://cdn.builder.io/api/v1/image/assets%2F4...
4,1212e5a7734743d683010c7c233894fd,Pink Camo Swatch,2024-10-02 18:16:02.219,NaT,published,Pink Camo,https://cdn.builder.io/api/v1/image/assets%2F4...


In [ ]:
print(f"\nDataFrame shape: {df_builder_filtered.shape}")


DataFrame shape: (8, 7)


In [ ]:
# Optionally, update the Google Sheet with this data
update_sheet(df_builder_filtered, 'Sheet4') # You might need to create a new sheet or adjust this

Data successfully uploaded to Google Sheets!


In [ ]:
!pip install requests-html

In [ ]:
!pip install lxml[html_clean]

In [ ]:
url = 'https://www.rothco.com/collection/womens-t-shirts-and-tank-tops-top-6?ss%5Bss_category_hierarchy%5D=Clothing'

In [ ]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup

# Start a session
session = HTMLSession()

# Make a request to the page
response = session.get(url)


In [ ]:
# response.html.arender()
response.html.arender(sleep=20, timeout=0)


<coroutine object HTML.arender at 0x7c855049a740>

In [ ]:
response.text

'<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8" />\n    <meta name="viewport" content="width=device-width, user-scalable=yes, initial-scale=1.0, maximum-scale=2.0" />\n    <script type="text/javascript">const Ziggy={"url":"https:\\/\\/www.rothco.com","port":null,"defaults":{},"routes":{"horizon.stats.index":{"uri":"horizon\\/api\\/stats","methods":["GET","HEAD"]},"horizon.workload.index":{"uri":"horizon\\/api\\/workload","methods":["GET","HEAD"]},"horizon.masters.index":{"uri":"horizon\\/api\\/masters","methods":["GET","HEAD"]},"horizon.monitoring.index":{"uri":"horizon\\/api\\/monitoring","methods":["GET","HEAD"]},"horizon.monitoring.store":{"uri":"horizon\\/api\\/monitoring","methods":["POST"]},"horizon.monitoring-tag.paginate":{"uri":"horizon\\/api\\/monitoring\\/{tag}","methods":["GET","HEAD"],"parameters":["tag"]},"horizon.monitoring-tag.destroy":{"uri":"horizon\\/api\\/monitoring\\/{tag}","methods":["DELETE"],"wheres":{"tag":".*"},"parameters":["tag"]},"ho

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
soup.find_all('img')

[]